In [ ]:
import threading
import time
import cv2
import numpy as np
from ultralytics import YOLO
import ipywidgets as widgets
from IPython.display import display

# Shared data between threads
shared_data = {
    "latest_frame": None,
    "depth_image": None,
    "frame_id": 0,
    "jpeg_image": None,
    "jpeg_depth": None,
}


# Data lock
lock = threading.Lock()

# Constants
FRAME_WIDTH = 672
IMAGE_CENTER_X = FRAME_WIDTH // 2


In [ ]:
model = YOLO("yolo11l_half.engine")

In [ ]:
import pyzed.sl as sl
import numpy as np
import cv2
r"""
class RealCamera:
    def __init__(self):
        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA  # 672x376
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print(f"Failed to open ZED camera: {status}")
            exit(1)

        self.runtime = sl.RuntimeParameters()
        self.image = sl.Mat()
        self.depth = sl.Mat()
        self.running = False
        self.frame_count = 0

    def start(self):
        self.running = True
        threading.Thread(target=self._capture_loop, daemon=True).start()

    def _capture_loop(self):
        global shared_data

        while self.running:
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)

                # Get data in numpy format
                color_image = self.image.get_data()
                depth_image = self.depth.get_data()

                # Convert BGRA to BGR
                color_image_bgr = cv2.cvtColor(color_image, cv2.COLOR_BGRA2BGR)

                with lock:
                    shared_data["latest_frame"] = color_image_bgr.copy()
                    shared_data["depth_image"] = np.copy(depth_image)
                    shared_data["frame_id"] = self.frame_count
                    self.frame_count += 1

            time.sleep(1 / 30)

    def stop(self):
        self.running = False
        self.zed.close()
"""

class RealCamera:
    def __init__(self):
        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA  # 672x376
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print(f"Failed to open ZED camera: {status}")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height

        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)

        self.running = False
        self.frame_count = 0

    def start(self):
        self.running = True
        threading.Thread(target=self._capture_loop, daemon=True).start()

    def _capture_loop(self):
        global shared_data

        while self.running:
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)

                color_image = self.image.get_data()
                color_image_bgr = cv2.cvtColor(color_image, cv2.COLOR_BGRA2BGR)
                depth_image = np.asanyarray(self.depth.get_data())

                # Process depth image
                depth_image = np.nan_to_num(depth_image, nan=0.0).astype(np.float32)
                depth_image[:94, :] = 0
                depth_image[282:, :] = 0
                depth_image[:, :168] = 0
                depth_image[:, 504:] = 0
                depth_image[depth_image < 100] = 0
                depth_image[depth_image > 1000] = 0

                # Create color depth map for display
                depth_colormap = cv2.applyColorMap(
                    cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET
                )

                # Resize for display
                scale = 0.1
                resized_image = cv2.resize(color_image_bgr, None, fx=scale, fy=scale)
                resized_depth_colormap = cv2.resize(depth_colormap, None, fx=scale, fy=scale)
                cv2.circle(resized_image, (int(self.width*scale//2), int(self.height*scale//2)), 1, (0, 255, 0), -1)

                # Encode to JPEG for streaming
                jpeg_image = self.bgr8_to_jpeg(resized_image)
                jpeg_depth = self.bgr8_to_jpeg(resized_depth_colormap)

                # Thread-safe update
                with lock:
                    shared_data["latest_frame"] = color_image_bgr.copy()
                    shared_data["depth_image"] = depth_image.copy()
                    shared_data["jpeg_image"] = jpeg_image
                    shared_data["jpeg_depth"] = jpeg_depth
                    shared_data["frame_id"] = self.frame_count
                    self.frame_count += 1

            time.sleep(1 / 30)  # 30 FPS

    def stop(self):
        self.running = False
        self.zed.close()

    def bgr8_to_jpeg(self, value):
        return bytes(cv2.imencode('.jpg', value)[1])


In [ ]:
class YOLODetector:
    def __init__(self, model_path="yolo11l_half.engine", distance_threshold=50, stale_threshold=5):
        self.model = YOLO(model_path)
        self.running = False
        self.next_id = 1  # Counter for assigning new person IDs
        self.distance_threshold = distance_threshold
        self.stale_threshold = stale_threshold

    def start(self):
        self.running = True
        threading.Thread(target=self._detection_loop, daemon=True).start()

    def stop(self):
        self.running = False

    def _get_center(self, bbox):
        x1, y1, x2, y2 = bbox
        return ((x1 + x2) / 2, (y1 + y2) / 2)

    def _euclidean_distance(self, c1, c2):
        return ((c1[0] - c2[0]) ** 2 + (c1[1] - c2[1]) ** 2) ** 0.5

    def _detection_loop(self):
        while self.running:
            frame = None
            frame_id = -1

            # Get the latest frame
            with lock:
                if shared_data["latest_frame"] is not None:
                    frame = shared_data["latest_frame"].copy()
                    frame_id = shared_data["frame_id"]
                    current_tracked = shared_data.get("tracked_people", {}).copy()
                else:
                    continue

            results = self.model(frame, verbose=False)
            new_detections = []

            # Gather new bounding boxes for people (class 0)
            for result in results:
                for i, cls_id in enumerate(result.boxes.cls):
                    if int(cls_id) == 0:
                        bbox = result.boxes.xyxy[i].cpu().numpy()
                        new_detections.append(bbox)

            updated_people = {}
            used_ids = set()

            # Match new detections to existing tracked people
            for bbox in new_detections:
                new_center = self._get_center(bbox)
                best_id = None
                best_dist = float("inf")

                for pid, pdata in current_tracked.items():
                    old_center = pdata["center"]
                    dist = self._euclidean_distance(new_center, old_center)
                    if dist < self.distance_threshold and pid not in used_ids and dist < best_dist:
                        best_id = pid
                        best_dist = dist

                if best_id is not None:
                    # Reuse the ID
                    updated_people[best_id] = {
                        "bbox": bbox,
                        "center": new_center,
                        "last_seen": frame_id
                    }
                    used_ids.add(best_id)
                else:
                    # Assign a new ID
                    new_id = self.next_id
                    self.next_id += 1
                    updated_people[new_id] = {
                        "bbox": bbox,
                        "center": new_center,
                        "last_seen": frame_id
                    }
                    used_ids.add(new_id)

            # Include still-valid unmatched tracked people (not seen in this frame but not too old)
            for pid, pdata in current_tracked.items():
                if pid not in used_ids:
                    if frame_id - pdata["last_seen"] <= self.stale_threshold:
                        updated_people[pid] = pdata

            # Save updated tracked people to shared data
            with lock:
                shared_data["tracked_people"] = updated_people

            time.sleep(0.05)


In [ ]:
camera = RealCamera()
detector = YOLODetector()

camera.start()
detector.start()

print("Camera and detection threads started.")

In [ ]:
STEERING_THRESHOLD = 30  # pixels
SAFE_DISTANCE_MM = 400
TOO_CLOSE_MM = 380

def steering_and_distance_logic_loop(interval=0.2):
    while True:
        with lock:
            frame = shared_data.get("latest_frame")
            depth_image = shared_data.get("depth_image")
            tracked_people = shared_data.get("tracked_people", {})
            selected_id = person_dropdown.value

        if selected_id is not None and frame is not None and depth_image is not None:
            person_data = tracked_people.get(int(selected_id))
            if person_data:
                x1, y1, x2, y2 = map(int, person_data["bbox"])
                cx = (x1 + x2) // 2
                cy = (y1 + y2) // 2

                delta = cx - IMAGE_CENTER_X

                if abs(delta) >= STEERING_THRESHOLD:
                    direction = "LEFT" if delta < 0 else "RIGHT"
                    print(f"[{selected_id}] STEER {direction}")
                else:
                    # Person is centered — now check distance
                    if 0 <= cy < depth_image.shape[0] and 0 <= cx < depth_image.shape[1]:
                        #window = depth_image[max(0, cy - 1):cy + 2, max(0, cx - 1):cx + 2]
                        #valid_depths = window[~np.isnan(window) & (window > 0)]
                        #if valid_depths.size > 0:
                        #    depth_val = np.median(valid_depths)
                        #    if depth_val > SAFE_DISTANCE_MM:
                        #        print(f"[{selected_id}]  CENTERED | {depth_val:.1f} mm — MOVE FORWARD ")
                        #    elif depth_val < TOO_CLOSE_MM:
                        #        print(f"[{selected_id}]  CENTERED | {depth_val:.1f} mm — MOVE BACKWARD ")
                        #    else:
                        #        print(f"[{selected_id}]  CENTERED | {depth_val:.1f} mm — HOLD POSITION ")
                        #else:
                        #    print(f"[{selected_id}]  CENTERED |  No valid depth in 3x3 region")
                        depth_val = depth_image[cy, cx]
                        if not np.isnan(depth_val) and depth_val > 0:
                            if depth_val > SAFE_DISTANCE_MM:
                                print(f"[{selected_id}]  CENTERED | {depth_val:.1f} mm — MOVE FORWARD ")
                            elif depth_val < TOO_CLOSE_MM:
                                print(f"[{selected_id}]  CENTERED | {depth_val:.1f} mm — MOVE BACKWARD ")
                            else:
                                print(f"[{selected_id}]  CENTERED | {depth_val:.1f} mm — HOLD POSITION ")
                        else:
                            print(f"[{selected_id}]  CENTERED |  Depth data unavailable")
                    else:
                        print(f"[{selected_id}]  CENTERED |  Invalid coordinates (cx={cx}, cy={cy})")
        time.sleep(interval)


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import cv2
import threading
import time

# --- UI Setup ---
display_widget = widgets.Image(format='jpeg', width=600)
person_dropdown = widgets.Dropdown(options=[], description="Person ID:")

display(widgets.VBox([display_widget, person_dropdown]))

# --- Utility ---
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

# --- Display Update Function ---
def update_display():
    with lock:
        frame = shared_data.get("latest_frame")
        frame_id = shared_data.get("frame_id", -1)
        tracked_people = shared_data.get("tracked_people", {})

    if frame is None:
        return

    display_frame = frame.copy()
    selected_id = str(person_dropdown.value)

    # Update dropdown options if people change
    current_ids = sorted(map(str, tracked_people.keys()))
    if person_dropdown.options != current_ids:
        person_dropdown.options = current_ids

    # Draw bounding boxes
    for pid, pdata in tracked_people.items():
        x1, y1, x2, y2 = map(int, pdata["bbox"])
        color = (0, 255, 0) if str(pid) == selected_id else (255, 0, 0)
        label = f"ID {pid}"
        cv2.rectangle(display_frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(display_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    #resized = cv2.resize(display_frame, None, fx=0.5, fy=0.5)
    #display_widget.value = bgr8_to_jpeg(resized)
    with lock:
        jpeg_image = shared_data.get("jpeg_image")
    
    if jpeg_image:
        display_widget.value = jpeg_image

# --- Periodic Update Loop ---
def schedule_update(interval=0.1):
    def loop():
        while True:
            update_display()
            time.sleep(interval)

    threading.Thread(target=loop, daemon=True).start()

# --- Start the Update Loop ---
schedule_update()
threading.Thread(target=steering_and_distance_logic_loop, daemon=True).start()

In [ ]:
camera.stop()
detector.stop()

print("Camera and detection threads stopped.")